In [2]:
import numpy as np
from scipy.optimize import root_scalar

def implied_probabilities(odds, grossmargin=0):
    """
    Calculate the implied probabilities from bookmaker odds in decimal format, while
    accounting for overround in the odds, using the balanced books method.

    Args:
        odds (numpy.array): A 1D or 2D array of bookmaker odds. The odds must be in decimal format.
        grossmargin (float): Must be 0 or greater. Default is 0.

    Returns:
        dict: A dictionary with the following keys:
            - 'probabilities': A numpy array of the implied probabilities.
            - 'bookmaker_margin': The bookmaker margin (aka the overround).
            - 'zvalues': The estimated amount of insider trading.

    References:
        - John Fingleton & Patrick Waldron (1999) Optimal Determination of Bookmakers' Betting Odds: Theory and Tests.
    """
    odds = np.asarray(odds)
    assert np.all(odds > 1), "All odds must be greater than 1."

    # Compute the inverse odds
    inverse_odds = 1 / odds

    # Compute the bookmaker margin (aka the overround)
    bookmaker_margin = np.sum(inverse_odds) - 1

    # Calculate the gross margin, which accounts for operating costs
    gross_margin = bookmaker_margin + grossmargin

    # Calculate the matrix B, which determines the size of the insiders' bet
    B = np.sum(inverse_odds, axis=0) - 1

    def balanced_books(x):
        # Compute the insiders' bet
        insider_bet = B * x / (1 + gross_margin * x)

        # Compute the total bet and the implied probabilities
        total_bet = np.sum(inverse_odds / (1 + insider_bet))
        probabilities = (inverse_odds / (1 + insider_bet)) / total_bet

        return np.sum(probabilities) - 1

    # Use scipy.optimize.root_scalar to find the value of x that satisfies the balanced books equation
    solution = root_scalar(balanced_books, method='brentq', bracket=(0, 1))

    # Compute the insiders' bet and the implied probabilities using the solution
    insider_bet = B * solution.root / (1 + gross_margin * solution.root)
    total_bet = np.sum(inverse_odds / (1 + insider_bet))
    probabilities = (inverse_odds / (1 + insider_bet)) / total_bet

    # Calculate the estimated amount of insider trading
    zvalues = insider_bet / (1 + insider_bet)

    return {'true odds': 1/probabilities, 'probabilities': probabilities, 'bookmaker_margin': bookmaker_margin, 'zvalues': zvalues}

my_odds = np.array([2.92,2.88,2.33])
implied_probabilities(my_odds)

{'true odds': array([3.26710777, 3.22235287, 2.60697298]),
 'probabilities': array([0.30608112, 0.31033224, 0.38358664]),
 'bookmaker_margin': 0.11887252500310286,
 'zvalues': 0.0}